In [1]:
!pip install builtwith

## 1. menambahkan abstrak bahasa inggris 

In [2]:
!pip install requests
!pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def ptaa():
    data = {"penulis": [], "judul": [], "pembimbing_pertama": [], "pembimbing_kedua": [], "abstrak_indonesia": [], "abstrak_inggris": []}

    for i in range(1, 4):
        url = "https://pta.trunojoyo.ac.id/c_search/byprod/10/{}".format(i)
        r = requests.get(url)
        request = r.content
        soup = BeautifulSoup(request, "html.parser")
        jurnals = soup.select('li[data-cat="#luxury"]')

        for jurnal in jurnals:
            response = requests.get(jurnal.select_one('a.gray.button')['href'])
            soup1 = BeautifulSoup(response.content, "html.parser")

            isi = soup1.select_one('div#content_journal')

            judul = isi.select_one('a.title').text

            penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1]
            # penulis = penulis_span.find_next('span').text.split(' : ')

            pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1]
            # pembimbing_pertama = pembimbing_pertama_span.find_next('span')

            pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(' :')[1]
            # pembimbing_kedua = pembimbing_kedua_span.find_next('span')

            # Ambil abstrak bahasa Indonesia
            abstrak_indonesia = ''
            abstrak_elements = isi.select('p[align="justify"]')
            if abstrak_elements:
                abstrak_indonesia = abstrak_elements[0].text

            # Ambil abstrak bahasa Inggris (mencari paragraf setelah abstrak bahasa Indonesia)
            abstrak_inggris = ''
            if abstrak_elements and len(abstrak_elements) > 1:
                 abstrak_inggris = abstrak_elements[1].text
            # if abstrak == '':
            #   abstrak = ' '.join(isi.find('p').findNext('p').stripped_strings).capitalize()

            data["penulis"].append(penulis)
            data["judul"].append(judul)
            data["pembimbing_pertama"].append(pembimbing_pertama)
            data["pembimbing_kedua"].append(pembimbing_kedua)
            data["abstrak_indonesia"].append(abstrak_indonesia)
            data["abstrak_inggris"].append(abstrak_inggris)


    df = pd.DataFrame(data)
    df.to_csv("pta.csv", index=False)

    return df

# Jalankan fungsi
ptaa()

C:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\soupsieve\css_parser.py:876: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


KeyboardInterrupt: 

In [4]:
import builtwith

# Analisis teknologi yang digunakan
res = builtwith.parse('https://pta.trunojoyo.ac.id')
print(res)

{'web-servers': ['Nginx'], 'javascript-frameworks': ['jQuery', 'jQuery UI']}


## 2 Menampilkan id berita, judul berita, kategori berita, dan isi berita

In [5]:
import urllib3

# Nonaktifkan SSL warning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def crawl_cnn_berita():
    data = {"idberita": [], "kategori_berita": [], "judul_berita": [], "isi_berita": []}
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/58.0.3029.110 Safari/537.3'
    }
    
    try:
        # Ambil halaman utama CNN Indonesia
        url = "https://www.cnnindonesia.com/"
        response = requests.get(url, headers=headers, verify=False)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Cari semua artikel di halaman utama
        articles = soup.find_all('article')
        processed_urls = set()
        
        for article in articles:
            link_tag = article.find('a', href=True)
            if not link_tag:
                continue
                
            berita_url = link_tag['href']
            if not berita_url.startswith('https://www.cnnindonesia.com/'):
                berita_url = 'https://www.cnnindonesia.com' + berita_url
            
            # Cek duplikasi
            if berita_url in processed_urls:
                continue
            processed_urls.add(berita_url)
            
            try:
                # Ambil detail berita
                berita_response = requests.get(berita_url, headers=headers, verify=False)
                berita_response.raise_for_status()
                berita_soup = BeautifulSoup(berita_response.content, "html.parser")
                
                # ID berita dari URL
                match = re.search(r'/([0-9]+)$', berita_url)
                idberita = match.group(1) if match else berita_url.split('/')[-1]
                
                # Judul berita
                judul_tag = berita_soup.find('h1')
                judul_berita = judul_tag.get_text(strip=True) if judul_tag else ''
                
                # Kategori berita
                kategori_tag = berita_soup.find('div', class_='breadcrumb')
                kategori_berita = ''
                if kategori_tag:
                    kategori_links = kategori_tag.find_all('a')
                    if kategori_links:
                        kategori_berita = kategori_links[-1].get_text(strip=True)
                
                # Isi berita: coba berbagai kemungkinan selector
                isi_berita = ''
                
                # Coba selector yang lebih umum
                selectors = [
                    'div.detail_text',
                    'div.detail__body-text', 
                    'div.detail-text',
                    'div[class*="detail"]',
                    'div[class*="content"]',
                    'div[class*="body"]',
                    'article div p',
                    '.article-content p',
                    '.post-content p'
                ]
                
                for selector in selectors:
                    isi_tag = berita_soup.select(selector)
                    if isi_tag:
                        if selector.endswith(' p'):
                            # Jika selector untuk paragraf
                            isi_berita = ' '.join([p.get_text(strip=True) for p in isi_tag])
                        else:
                            # Jika selector untuk div
                            isi_berita = isi_tag[0].get_text(separator=' ', strip=True)
                        break
                
                # Jika masih kosong, coba ambil semua paragraf dalam body
                if not isi_berita:
                    paragraphs = berita_soup.find_all('p')
                    if paragraphs:
                        # Filter paragraf yang memiliki teks panjang (kemungkinan isi berita)
                        long_paragraphs = [p.get_text(strip=True) for p in paragraphs if len(p.get_text(strip=True)) > 50]
                        isi_berita = ' '.join(long_paragraphs[:5])  
                data["idberita"].append(idberita)
                data["kategori_berita"].append(kategori_berita)
                data["judul_berita"].append(judul_berita)
                data["isi_berita"].append(isi_berita)
                
            except Exception as e:
                print(f'Gagal mengambil berita: {berita_url} | Error: {e}')
    
    except Exception as e:
        print(f'Terjadi kesalahan: {e}')
    
    df = pd.DataFrame(data)
    df.to_csv("cnn_berita.csv", index=False)
    
    # display(df)
    return df

# jalankan scraping
crawl_cnn_berita()

,idberita,kategori_berita,judul_berita,isi_berita
0,www.cnnindonesia.com#,,Kapal Bantuan Gaza GSF Diserang Drone di Tunis...,BERITA TERBARU
1,milisi-palestina-ledakkan-tank-israel-4-tentar...,Timur Tengah,"Milisi Palestina Ledakkan Tank Israel, 4 Tenta...","Jakarta, CNN Indonesia -- Sebanyak tiga milisi..."
2,kapal-bawa-bantuan-gaza-diserang-drone-di-tuni...,Timur Tengah,Kapal Bawa Bantuan Gaza Diserang Drone di Tuni...,"Jakarta, CNN Indonesia -- Salah satu kapal yan..."
3,video-gen-z-nepal-demo-gegara-pemerintah-bloki...,Asia Pasifik,VIDEO: Gen-Z Nepal Demo Gegara Pemerintah Blok...,"Jakarta, CNN Indonesia -- Kerusuhan besar terj..."
4,video-momen-pemakaman-giorgio-armani-yang-dige...,Trends,VIDEO: Momen Pemakaman Giorgio Armani yang Dig...,"Jakarta, CNN Indonesia -- Desainer Italia, Gio..."
5,daftar-susunan-pemain-timnas-indonesia-u-23-vs...,Sepakbola,Daftar Susunan Pemain Timnas Indonesia U-23 vs...,"Jakarta, CNN Indonesia -- Timnas Indonesia U-2..."
6,video-peneliti-inggris-ciptakan-suplemen-baru-...,Sains,VIDEO: Peneliti Inggris Ciptakan Suplemen Baru...,"Jakarta, CNN Indonesia -- Peneliti Universitas..."
7,menhan-sjafrie-respons-tindakan-tni-bidik-duga...,Hukum Kriminal,Menhan Sjafrie Respons Tindakan TNI Bidik Duga...,"Jakarta, CNN Indonesia -- Menteri Pertahanan (..."
8,lisa-mariana-surati-bareskrim-minta-tes-dna-ul...,Hukum Kriminal,"Lisa Mariana Surati Bareskrim, Minta Tes DNA U...","Jakarta, CNN Indonesia -- Selebgram Lisa Maria..."
9,didemo-ribuan-gen-z-pm-nepal-kp-sharma-oli-mundur,Asia Pasifik,"Didemo Ribuan Gen Z, PM Nepal KP Sharma Oli Mu...","Jakarta, CNN Indonesia -- Perdana Menteri Nepa..."


In [7]:
import requests
from bs4 import BeautifulSoup
import re

def crawl_cnn_article(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/58.0.3029.110 Safari/537.3'
    }
    
    try:
        response = requests.get(url, headers=headers, verify=False)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Ambil judul berita dari tag h1
        judul_tag = soup.find('h1')
        judul_berita = judul_tag.get_text(strip=True) if judul_tag else 'Judul tidak ditemukan'
        
        # Ambil kategori berita dari breadcrumb
        kategori_berita = 'Kategori tidak ditemukan'
        breadcrumb = soup.find('div', class_='breadcrumb')
        if breadcrumb:
            kategori_links = breadcrumb.find_all('a')
            if kategori_links:
                kategori_berita = kategori_links[-1].get_text(strip=True)
        
        # Ambil isi berita dengan berbagai selector
        isi_berita = ''
        selectors = [
            'div.detail_text',
            'div.detail__body-text', 
            'div.detail-text',
            'div[class*="detail"]',
            'div[class*="content"]',
            'div[class*="body"]'
        ]
        
        for selector in selectors:
            content_div = soup.select_one(selector)
            if content_div:
                # Ambil semua paragraf dalam div tersebut
                paragraphs = content_div.find_all('p')
                if paragraphs:
                    isi_berita = ' '.join([p.get_text(strip=True) for p in paragraphs])
                else:
                    isi_berita = content_div.get_text(separator=' ', strip=True)
                break
        
        # Jika masih kosong, coba ambil semua paragraf di halaman
        if not isi_berita:
            all_paragraphs = soup.find_all('p')
            if all_paragraphs:
                # Filter paragraf yang memiliki teks panjang
                long_paragraphs = [p.get_text(strip=True) for p in all_paragraphs 
                                 if len(p.get_text(strip=True)) > 50]
                isi_berita = ' '.join(long_paragraphs[:5])
        
        if not isi_berita:
            isi_berita = 'Isi berita tidak ditemukan'
        
        # Tampilkan hasil
        print("=" * 80)
        print("HASIL CRAWLING BERITA CNN INDONESIA")
        print("=" * 80)
        print(f"Judul Berita: {judul_berita}")
        print(f"Kategori: {kategori_berita}")
        print("-" * 80)
        print("Isi Berita:")
        print(isi_berita[:500] + "..." if len(isi_berita) > 500 else isi_berita)
        print("=" * 80)
        
        return {
            'judul': judul_berita,
            'kategori': kategori_berita,
            'isi': isi_berita
        }
        
    except requests.exceptions.RequestException as e:
        print(f"Terjadi kesalahan saat mengakses {url}: {e}")
        return None

# Contoh penggunaan dengan URL berita CNN Indonesia
url_berita = "https://www.cnnindonesia.com/olahraga/20250907140830-156-1270914/start-dari-belakang-bagnaia-ngarep-finis-di-10-besar-motogp-catalunya"
result = crawl_cnn_article(url_berita)

HASIL CRAWLING BERITA CNN INDONESIA
Judul Berita: Start dari Belakang, Bagnaia Ngarep Finis di 10 Besar MotoGP Catalunya
Kategori: Moto GP
--------------------------------------------------------------------------------
Isi Berita:
Pembalap DucatiFrancesco Bagnaiaberharap finis di 10 besarMotoGP Catalunya 2025di Sirkuit Catalunya saat start dari posisi ke-21. Start ke-21 dalam MotoGP Catalunya 2025 jadi yang terburuk bagi Bagnaia setelah MotoGP Portugal 2022. Pada tiga tahun lalu tersebut Bagnaia start dari posisi ke-25 atau paling buncit. Bagnaia mengaku sudah memprediksi kesulitan tampil bagus dalam MotoGP Catalunya 2025 setelah finis ke-14 di MotoGP Hungaria. ADVERTISEMENT SCROLL TO CONTINUE WITH CONTENT "Berjuang. Sepe...


## 3. crawl link keluar dari sebuah web

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
# import networkx as nx
# import matplotlib.pyplot as plt
from urllib.parse import urljoin, urlparse

In [9]:
# Variabel global
visited = set()
web_structur = []

In [10]:
def crawl_website(url, base_domain, max_depth=2, depth=0):
    if url in visited or depth > max_depth:
        return
    visited.add(url)

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Ambil semua link
        links = soup.find_all('a', href=True)
        for link in links:
            full_url = urljoin(url, link['href'])  # absolute URL

            web_structur.append({
                "page": url,
                "link keluar": full_url
            })

            # Batasi hanya ke domain informatika.trunojoyo.ac.id
            if base_domain in urlparse(full_url).netloc:
                crawl_website(full_url, base_domain, max_depth, depth+1)

    except Exception as e:
        print(f"Error saat akses {url}: {e}")

def run_crawler(start_url, max_depth=2):
    base_domain = urlparse(start_url).netloc
    crawl_website(start_url, base_domain, max_depth=max_depth)

In [11]:
start_url = "https://pta.trunojoyo.ac.id"
run_crawler(start_url, max_depth=1)

print(f"Jumlah data hasil crawl: {len(web_structur)}")

Error saat akses https://pta.trunojoyo.ac.id/index.html: 404 Client Error: Not Found for url: https://pta.trunojoyo.ac.id/index.html
Jumlah data hasil crawl: 5249


In [12]:
df = pd.DataFrame(web_structur)
df.to_csv("hasil_crawl.csv", index=False, encoding="utf-8")

print("Hasil crawling disimpan ke 'hasil_crawl.csv'")
df.head(10)

Hasil crawling disimpan ke 'hasil_crawl.csv'


,page,link keluar
0,https://pta.trunojoyo.ac.id,https://pta.trunojoyo.ac.id/index.html
1,https://pta.trunojoyo.ac.id,https://pta.trunojoyo.ac.id
2,https://pta.trunojoyo.ac.id,https://pta.trunojoyo.ac.id/
3,https://pta.trunojoyo.ac.id/,https://pta.trunojoyo.ac.id/index.html
4,https://pta.trunojoyo.ac.id/,https://pta.trunojoyo.ac.id/
5,https://pta.trunojoyo.ac.id/,https://pta.trunojoyo.ac.id/
6,https://pta.trunojoyo.ac.id/,https://pta.trunojoyo.ac.id/c_search/
7,https://pta.trunojoyo.ac.id/,https://pta.trunojoyo.ac.id/c_template/
8,https://pta.trunojoyo.ac.id/,https://library.trunojoyo.ac.id/detil.php?id=23
9,https://pta.trunojoyo.ac.id/,https://pta.trunojoyo.ac.id/c_contact/
